In [2]:
### Imports
import numpy as np 
import pandas as pd 
import sys
pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings("ignore")
import gc

In [3]:
### Import datasets
train = pd.read_csv('data/application_train.csv')
bureau = pd.read_csv('data/bureau.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')
credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
installments_payments = pd.read_csv('data/installments_payments.csv')
POS_CASH_balance = pd.read_csv('data/POS_CASH_balance.csv')
previous_application = pd.read_csv('data/previous_application.csv')

In [4]:
### Bureau Transactions of single customer (SK_ID_CURR = 100001)

bureau[bureau['SK_ID_CURR'] == 100001]

SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
248484  100001      5896630       Closed        currency 1     -857           
248485  100001      5896631       Closed        currency 1     -909           
248486  100001      5896632       Closed        currency 1     -879           
248487  100001      5896633       Closed        currency 1     -1572          
248488  100001      5896634       Active        currency 1     -559           
248489  100001      5896635       Active        currency 1     -49            
248490  100001      5896636       Active        currency 1     -320           

        CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
248484  0                  -492.0               -553.0               
248485  0                  -179.0               -877.0               
248486  0                  -514.0               -544.0               
248487  0                  -1329.0              -1328.0              
248488  0                   902.0               NaN                  
248489  0                   1778.0              NaN                  
248490  0                   411.0               NaN                  

        AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
248484 NaN                      0                   112500.0         
248485 NaN                      0                   279720.0         
248486 NaN                      0                   91620.0          
248487 NaN                      0                   85500.0          
248488 NaN                      0                   337680.0         
248489 NaN                      0                   378000.0         
248490 NaN                      0                   168345.0         

        AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
248484  0.0                  0.0                   0.0                      
248485  0.0                  0.0                   0.0                      
248486  0.0                  0.0                   0.0                      
248487  0.0                  0.0                   0.0                      
248488  113166.0             0.0                   0.0                      
248489  373239.0             0.0                   0.0                      
248490  110281.5            NaN                    0.0                      

            CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  
248484  Consumer credit -155                 0.0          
248485  Consumer credit -155                 0.0          
248486  Consumer credit -155                 0.0          
248487  Consumer credit -155                 0.0          
248488  Consumer credit -6                   4630.5       
248489  Consumer credit -16                  10822.5      
248490  Consumer credit -10                  9364.5

In [5]:
### Visualize number of customers
train['SK_ID_CURR'].describe()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0       100002      1       Cash loans         M           N             
1       100003      0       Cash loans         F           N             
2       100004      0       Revolving loans    M           Y             
3       100006      0       Cash loans         F           N             
4       100007      0       Cash loans         M           N             
5       100008      0       Cash loans         M           N             
6       100009      0       Cash loans         F           Y             
7       100010      0       Cash loans         M           Y             
8       100011      0       Cash loans         F           N             
9       100012      0       Revolving loans    M           N             
10      100014      0       Cash loans         F           N             
11      100015      0       Cash loans         F           N             
12      100016      0       Cash loans         F           N             
13      100017      0       Cash loans         M           Y             
14      100018      0       Cash loans         F           N             
15      100019      0       Cash loans         M           Y             
16      100020      0       Cash loans         M           N             
17      100021      0       Revolving loans    F           N             
18      100022      0       Revolving loans    F           N             
19      100023      0       Cash loans         F           N             
20      100024      0       Revolving loans    M           Y             
21      100025      0       Cash loans         F           Y             
22      100026      0       Cash loans         F           N             
23      100027      0       Cash loans         F           N             
24      100029      0       Cash loans         M           Y             
25      100030      0       Cash loans         F           N             
26      100031      1       Cash loans         F           N             
27      100032      0       Cash loans         M           N             
28      100033      0       Cash loans         M           Y             
29      100034      0       Revolving loans    M           N             
...        ...     ..                   ...   ..          ..             
307481  456225      1       Cash loans         M           N             
307482  456226      0       Cash loans         F           Y             
307483  456227      0       Cash loans         F           N             
307484  456228      0       Cash loans         F           Y             
307485  456229      0       Revolving loans    F           N             
307486  456230      0       Cash loans         F           Y             
307487  456231      0       Cash loans         M           N             
307488  456232      0       Cash loans         F           N             
307489  456233      1       Cash loans         F           N             
307490  456234      0       Cash loans         M           N             
307491  456235      0       Cash loans         M           Y             
307492  456236      0       Cash loans         M           Y             
307493  456237      0       Cash loans         F           N             
307494  456238      0       Cash loans         M           Y             
307495  456239      0       Cash loans         M           Y             
307496  456240      0       Revolving loans    F           N             
307497  456241      0       Cash loans         F           Y             
307498  456242      0       Cash loans         M           Y             
307499  456243      0       Cash loans         F           N             
307500  456244      0       Cash loans         F           N             
307501  456245      0       Cash loans         F           N             
307502  456246      0       Cash loans         F           N             
307503  4562

In [19]:
### Feature Engineering 1
### Number of Past Loan per Customer

grp = bureau[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
B = bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(B.shape)

(1716428, 18)


In [20]:
### Feature Engineering 2
### Number of Types of Past Loans per Customer

grp = B[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
B

SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0        215354      5714462       Closed        currency 1     -497           
1        215354      5714463       Active        currency 1     -208           
2        215354      5714464       Active        currency 1     -203           
3        215354      5714465       Active        currency 1     -203           
4        215354      5714466       Active        currency 1     -629           
5        215354      5714467       Active        currency 1     -273           
6        215354      5714468       Active        currency 1     -43            
7        162297      5714469       Closed        currency 1     -1896          
8        162297      5714470       Closed        currency 1     -1146          
9        162297      5714471       Active        currency 1     -1146          
10       162297      5714472       Active        currency 1     -1146          
11       162297      5714473       Closed        currency 1     -2456          
12       162297      5714474       Active        currency 1     -277           
13       402440      5714475       Active        currency 1     -96            
14       238881      5714482       Closed        currency 1     -318           
15       238881      5714484       Closed        currency 1     -2911          
16       238881      5714485       Closed        currency 1     -2148          
17       238881      5714486       Active        currency 1     -381           
18       238881      5714487       Active        currency 1     -95            
19       238881      5714488       Closed        currency 1     -444           
20       238881      5714489       Active        currency 1     -392           
21       222183      5714491       Active        currency 1     -784           
22       222183      5714492       Active        currency 1     -774           
23       222183      5714493       Active        currency 1     -395           
24       222183      5714495       Closed        currency 1     -2744          
25       222183      5714496       Closed        currency 1     -1103          
26       222183      5714497       Active        currency 1     -315           
27       426155      5714498       Closed        currency 1     -1331          
28       426155      5714499       Closed        currency 1     -2534          
29       426155      5714500       Closed        currency 1     -845           
...         ...          ...          ...               ...      ...           
1716398  312021      5057541       Closed        currency 1     -1529          
1716399  232638      5057553       Active        currency 1     -613           
1716400  318662      5057561       Active        currency 1     -815           
1716401  145715      5057569       Closed        currency 1     -2333          
1716402  352358      5057580       Closed        currency 1     -1714          
1716403  361928      5057589       Active        currency 1     -765           
1716404  166279      5057599       Active        currency 1     -99            
1716405  438109      5057605       Closed        currency 1     -2390          
1716406  335468      5057618       Closed        currency 1     -2517          
1716407  146896      5057629       Closed        currency 1     -1990          
1716408  146896      5057632       Closed        currency 1     -434           
1716409  146896      5057636       Closed        currency 1     -1705          
1716410  280228      5057642       Active        currency 1     -575           
1716411  232118      5057652       Closed        currency 1     -821           
1716412  288038      5057657       Active        currency 1     -541           
1716413  288038      5057665       Closed        currency 1     -463           
1716414  445222      5057676       Closed        currency 1     -1628          
1716415  317217      5057684       Closed        currency 1     -1970          
1716416  

In [21]:
### Feature Engineering 3
### Average Number of Past Loans per Type per Customer

B['AVERAGE_LOAN_TYPE'] = B['BUREAU_LOAN_COUNT']/B['BUREAU_LOAN_TYPES']
del B['BUREAU_LOAN_COUNT'], B['BUREAU_LOAN_TYPES']
gc.collect()

138

In [22]:
### Feature Engineering 4
### Percent of Active Loans from Buereau Data

# Create a new dummy column for whether CREDIT is ACTIVE OR CLOED 
B['CREDIT_ACTIVE_BINARY'] = B['CREDIT_ACTIVE']

def f(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

B['CREDIT_ACTIVE_BINARY'] = B.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
grp = B.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del B['CREDIT_ACTIVE_BINARY']
gc.collect()
print(B.shape)

B[B['SK_ID_CURR'] == 100001]

(1716428, 19)


SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
248484  100001      5896630       Closed        currency 1     -857           
248485  100001      5896631       Closed        currency 1     -909           
248486  100001      5896632       Closed        currency 1     -879           
248487  100001      5896633       Closed        currency 1     -1572          
248488  100001      5896634       Active        currency 1     -559           
248489  100001      5896635       Active        currency 1     -49            
248490  100001      5896636       Active        currency 1     -320           

        CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
248484  0                  -492.0               -553.0               
248485  0                  -179.0               -877.0               
248486  0                  -514.0               -544.0               
248487  0                  -1329.0              -1328.0              
248488  0                   902.0               NaN                  
248489  0                   1778.0              NaN                  
248490  0                   411.0               NaN                  

        AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
248484 NaN                      0                   112500.0         
248485 NaN                      0                   279720.0         
248486 NaN                      0                   91620.0          
248487 NaN                      0                   85500.0          
248488 NaN                      0                   337680.0         
248489 NaN                      0                   378000.0         
248490 NaN                      0                   168345.0         

        AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
248484  0.0                  0.0                   0.0                      
248485  0.0                  0.0                   0.0                      
248486  0.0                  0.0                   0.0                      
248487  0.0                  0.0                   0.0                      
248488  113166.0             0.0                   0.0                      
248489  373239.0             0.0                   0.0                      
248490  110281.5            NaN                    0.0                      

            CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  AVERAGE_LOAN_TYPE  \
248484  Consumer credit -155                 0.0          7.0                 
248485  Consumer credit -155                 0.0          7.0                 
248486  Consumer credit -155                 0.0          7.0                 
248487  Consumer credit -155                 0.0          7.0                 
248488  Consumer credit -6                   4630.5       7.0                 
248489  Consumer credit -16                  10822.5      7.0                 
248490  Consumer credit -10                  9364.5       7.0                 

        ACTIVE_LOANS_PERCENTAGE  
248484  0.428571                 
248485  0.428571                 
248486  0.428571                 
248487  0.428571                 
248488  0.428571                 
248489  0.428571                 
248490  0.428571

In [23]:
### Feature Engineering 5
### Num of Days Between Successive Past Applications for Each Customer

# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
grp = B[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)
print("Grouping and Sorting done")

# Calculate Difference between the number of Days 
grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT']*-1
grp1['DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT1'], grp1['DAYS_CREDIT'], grp1['SK_ID_CURR']
gc.collect()
print("Difference days calculated")

B = B.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
print("Difference in Dates between Previous CB applications is CALCULATED ")
print(B.shape)

grp = B.groupby(by = ['SK_ID_CURR'])
B = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)
B[B['SK_ID_CURR'] == 100001]

Grouping and Sorting done
Difference days calculated
Difference in Dates between Previous CB applications is CALCULATED 
(1716428, 20)


SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0  100001      5896635       Active        currency 1     -49            
1  100001      5896636       Active        currency 1     -320           
2  100001      5896634       Active        currency 1     -559           
3  100001      5896630       Closed        currency 1     -857           
4  100001      5896632       Closed        currency 1     -879           
5  100001      5896631       Closed        currency 1     -909           
6  100001      5896633       Closed        currency 1     -1572          

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0  0                   1778.0              NaN                  
1  0                   411.0               NaN                  
2  0                   902.0               NaN                  
3  0                  -492.0               -553.0               
4  0                  -514.0               -544.0               
5  0                  -179.0               -877.0               
6  0                  -1329.0              -1328.0              

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0 NaN                      0                   378000.0         
1 NaN                      0                   168345.0         
2 NaN                      0                   337680.0         
3 NaN                      0                   112500.0         
4 NaN                      0                   91620.0          
5 NaN                      0                   279720.0         
6 NaN                      0                   85500.0          

   AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
0  373239.0             0.0                   0.0                      
1  110281.5            NaN                    0.0                      
2  113166.0             0.0                   0.0                      
3  0.0                  0.0                   0.0                      
4  0.0                  0.0                   0.0                      
5  0.0                  0.0                   0.0                      
6  0.0                  0.0                   0.0                      

       CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  AVERAGE_LOAN_TYPE  \
0  Consumer credit -16                  10822.5      7.0                 
1  Consumer credit -10                  9364.5       7.0                 
2  Consumer credit -6                   4630.5       7.0                 
3  Consumer credit -155                 0.0          7.0                 
4  Consumer credit -155                 0.0          7.0                 
5  Consumer credit -155                 0.0          7.0                 
6  Consumer credit -155                 0.0          7.0                 

   ACTIVE_LOANS_PERCENTAGE  DAYS_DIFF  
0  0.428571                 0          
1  0.428571                 271        
2  0.428571                 239        
3  0.428571                 298        
4  0.428571                 22         
5  0.428571                 30         
6  0.428571                 663

In [12]:
B[B['SK_ID_CURR'] == 100001]

SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
248484  100001      5896630       Closed        currency 1     -857           
248485  100001      5896631       Closed        currency 1     -909           
248486  100001      5896632       Closed        currency 1     -879           
248487  100001      5896633       Closed        currency 1     -1572          
248488  100001      5896634       Active        currency 1     -559           
248489  100001      5896635       Active        currency 1     -49            
248490  100001      5896636       Active        currency 1     -320           

        CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
248484  0                  -492.0               -553.0               
248485  0                  -179.0               -877.0               
248486  0                  -514.0               -544.0               
248487  0                  -1329.0              -1328.0              
248488  0                   902.0               NaN                  
248489  0                   1778.0              NaN                  
248490  0                   411.0               NaN                  

        AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
248484 NaN                      0                   112500.0         
248485 NaN                      0                   279720.0         
248486 NaN                      0                   91620.0          
248487 NaN                      0                   85500.0          
248488 NaN                      0                   337680.0         
248489 NaN                      0                   378000.0         
248490 NaN                      0                   168345.0         

        AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
248484  0.0                  0.0                   0.0                      
248485  0.0                  0.0                   0.0                      
248486  0.0                  0.0                   0.0                      
248487  0.0                  0.0                   0.0                      
248488  113166.0             0.0                   0.0                      
248489  373239.0             0.0                   0.0                      
248490  110281.5            NaN                    0.0                      

            CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  AVERAGE_LOAN_TYPE  \
248484  Consumer credit -155                 0.0          7.0                 
248485  Consumer credit -155                 0.0          7.0                 
248486  Consumer credit -155                 0.0          7.0                 
248487  Consumer credit -155                 0.0          7.0                 
248488  Consumer credit -6                   4630.5       7.0                 
248489  Consumer credit -16                  10822.5      7.0                 
248490  Consumer credit -10                  9364.5       7.0                 

        ACTIVE_LOANS_PERCENTAGE  DAYS_DIFF  
248484  0.428571                 298        
248485  0.428571                 30         
248486  0.428571                 22         
248487  0.428571                 663        
248488  0.428571                 239        
248489  0.428571                 0          
248490  0.428571                 271